MAIN CODE


In [4]:
# -*- coding: utf-8 -*-
"""Untitled4.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1AT_rbhDvfCM7lYmCAbLiMYJH6FL1qPrp
"""
##
!pip install torch transformers faiss-cpu sentence-transformers pdfplumber

!pip install pymupdf

!pip install python-docx

!pip install pytesseract

!pip install openai-whisper

!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install faster-whisper
!pip install torch

!pip install beautifulsoup4 requests youtube_transcript_api pytube openai-whisper moviepy

import os
import re
import faiss
import requests
import pdfplumber
import numpy as np
import pandas as pd
import pytesseract
import faster_whisper
from bs4 import BeautifulSoup
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
from PIL import Image
from docx import Document
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import mimetypes
import nltk
import torch

from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

# Load Summarization Model (Mistral-7B-Instruct via Hugging Face)
print("🔹 Loading Summarization Model...")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn").to("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")


# Load Sentence Embedding Model (Efficient Retrieval)
print("🔹 Loading Embedding Model...")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load Faster Whisper for Video Transcription
print("🔹 Loading Whisper Model...")
whisper_model = faster_whisper.WhisperModel("base")

# FAISS Index Initialization
dimension = 384  # MiniLM embedding size
faiss_index = faiss.IndexFlatL2(dimension)
documents = []  # Stores extracted sentences

# 🟢 Function to Extract Text from Web Pages
def extract_text_from_web(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract text from paragraphs
        paragraphs = soup.find_all('p')
        text = "\n".join([para.get_text() for para in paragraphs])

        return text.strip() if text.strip() else "No meaningful text found on the webpage."

    except Exception as e:
        return f"Error extracting text from {url}: {str(e)}"

# 🟢 Function to Extract Subtitles from YouTube Videos
def extract_text_from_youtube(url):
    try:
        video_id = url.split("v=")[-1]
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        text = " ".join([t['text'] for t in transcript])

        return text.strip() if text.strip() else "No subtitles found for this YouTube video."

    except Exception as e:
        return f"Error extracting subtitles from YouTube: {str(e)}"

# 🟢 Function to Process File or URL
def process_input(source):
    if source.startswith("http"):
        if "youtube.com" in source or "youtu.be" in source:
            extracted_text = extract_text_from_youtube(source)
        else:
            extracted_text = extract_text_from_web(source)
    else:
        extracted_text = process_file(source)

    if isinstance(extracted_text, str) and extracted_text.strip():
        return extracted_text
    return None

# 🟢 Function to Extract and Clean Text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"

    # Clean extracted text
    text = re.sub(r"\n+", "\n", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"[0-9]+\.\d+|\d+\.\s?", "", text)  # Remove section numbers
    return text.strip()

# 🟢 Function to Extract Text from DOCX
def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text.strip()

from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import pytesseract

# Load BLIP model and processor globally (to avoid reloading for every image)
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def extract_text_from_image(file_path):
    text = pytesseract.image_to_string(Image.open(file_path)).strip()

    if text:
        return text  # ✅ Return a string instead of a dictionary

    print(f"⚠️ No readable text found in {file_path}. Using vision model (BLIP).")

    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    image = Image.open(file_path).convert("RGB")
    inputs = processor(image, return_tensors="pt")
    output = model.generate(**inputs)
    caption = processor.decode(output[0], skip_special_tokens=True)

    return f"[Vision Model]: {caption}"  # ✅ Always return a string

import os
import moviepy.editor as mp
import whisper
def extract_text_from_video(file_path):
    try:
        audio_path = "temp_audio.wav"
        video = mp.VideoFileClip(file_path)
        video.audio.write_audiofile(audio_path)

        model = whisper.load_model("base")
        result = model.transcribe(audio_path)

        os.remove(audio_path)
        return result['text'] if result['text'].strip() else "No meaningful audio detected."

    except Exception as e:
        return f"Error processing video: {str(e)}"

# 🟢 Function to Add Extracted Text to FAISS Index
def add_document(text):
    global faiss_index, documents
    sentences = sent_tokenize(text)  # Better sentence splitting
    embeddings = embedding_model.encode(sentences, convert_to_numpy=True)
    faiss_index.add(embeddings)
    documents.extend(sentences)

def process_file(file_path):
    mime_type, _ = mimetypes.guess_type(file_path)

    if not mime_type:
        print(f"❌ Could not determine file type: {file_path}")
        return None

    if "pdf" in mime_type:
        extracted_text = extract_text_from_pdf(file_path)
    elif "word" in mime_type or file_path.lower().endswith(".docx"):
        extracted_text = extract_text_from_docx(file_path)
    elif "csv" in mime_type:
        extracted_text = process_csv(file_path)
    elif mime_type.startswith("image"):
        extracted_text = extract_text_from_image(file_path)
    elif mime_type.startswith("video") or mime_type.startswith("audio"):
        extracted_text = extract_text_from_video(file_path)
    else:
        print(f"❌ Unsupported file type: {mime_type}")
        return None

    # ✅ Fix: Check if extracted_text is a dictionary before calling strip()
    if isinstance(extracted_text, dict):
        if "Extracted Text" in extracted_text and extracted_text["Extracted Text"].strip():
            return extracted_text
        else:
            print(f"⚠️ No meaningful text extracted from {file_path}. Returning description.")
            return extracted_text  # Return dictionary response as is

    elif isinstance(extracted_text, str):
        return extracted_text.strip() if extracted_text.strip() else None

    return None  # Handle unexpected cases

import pandas as pd

def process_csv(file_path):
    df = pd.read_csv(file_path)

    summary = f"🔹 CSV File Summary:\n"
    summary += f"- Total Rows: {len(df)}\n"
    summary += f"- Total Columns: {len(df.columns)}\n"
    summary += f"- Column Names: {', '.join(df.columns)}\n"

    # Column Data Types
    col_types = df.dtypes.astype(str).to_dict()
    summary += f"- Column Types:\n" + "\n".join([f"  • {col}: {dtype}" for col, dtype in col_types.items()]) + "\n"

    # Numeric Summary
    num_cols = df.select_dtypes(include=['number']).columns
    if len(num_cols) > 0:
        num_summary = df[num_cols].describe().to_dict()
        summary += "\n🔹 Numerical Summary:\n"
        for col, stats in num_summary.items():
            summary += f"  • {col}: {stats}\n"

    # Categorical Summary
    cat_cols = df.select_dtypes(include=['object']).columns
    if len(cat_cols) > 0:
        summary += "\n🔹 Categorical Summary:\n"
        for col in cat_cols:
            top_values = df[col].value_counts().head(3).to_dict()  # Show top 3 frequent values
            summary += f"  • {col}: {top_values}\n"

    return summary  # ✅ Always return a string instead of a dictionary

def retrieve_top_k(query, k=3):
    if not documents:
        return "❌ No documents available for retrieval."

    query_embedding = embedding_model.encode([query], convert_to_numpy=True)
    _, indices = faiss_index.search(query_embedding, k)

    retrieved_texts = [documents[idx] for idx in indices[0] if idx < len(documents)]

    if not retrieved_texts:
        return "❌ No relevant context found."

    return "\n".join(retrieved_texts)

def generate_summary(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)

    with torch.no_grad():  # Disable gradients for faster inference
        summary_ids = model.generate(
            inputs.input_ids,
            max_length=150,
            min_length=50,
            num_beams=4,
            early_stopping=True
        )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 🔹 RAG Chatbot with File & Web Scraping Support
def chatbot():
    global faiss_index, documents

    faiss_index = faiss.IndexFlatL2(dimension)
    documents = []

    print("\n🔹 RAG Chatbot - Upload Files or URLs")

    while True:
        source = input("Enter file path or URL (or 'done' to finish): ").strip()
        if source.lower() == "done":
            break

        print(f"✅ Processing {source} ...")
        extracted_text = process_input(source)

        if extracted_text:
            add_document(extracted_text)
            print(f"✅ Successfully processed {source}.")
        else:
            print(f"❌ Failed to process {source}.")

    print("\n✅ Index Built Successfully!\n")

    while True:
        query = input("\n🔍 Enter your query (or 'exit' to quit): ").strip()
        if query.lower() == "exit":
            break

        retrieved_text = retrieve_top_k(query)
        print("\n📌 **Top Retrieved Contexts:**")
        print(retrieved_text)

        print("\n📌 **Abstractive Summary:**")
        summary = generate_summary(retrieved_text)
        print(summary)




Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


🔹 Loading Summarization Model...
🔹 Loading Embedding Model...
🔹 Loading Whisper Model...


CSV FILE INPUT


In [5]:
if __name__ == "__main__":
    chatbot()



🔹 RAG Chatbot - Upload Files or URLs
Enter file path or URL (or 'done' to finish): /content/Students_Grading_Dataset.csv
✅ Processing /content/Students_Grading_Dataset.csv ...
✅ Successfully processed /content/Students_Grading_Dataset.csv.
Enter file path or URL (or 'done' to finish): done

✅ Index Built Successfully!


🔍 Enter your query (or 'exit' to quit): describe oit

📌 **Top Retrieved Contexts:**
🔹 CSV File Summary:
- Total Rows: 5000
- Total Columns: 23
- Column Names: Student_ID, First_Name, Last_Name, Email, Gender, Age, Department, Attendance (%), Midterm_Score, Final_Score, Assignments_Avg, Quizzes_Avg, Participation_Score, Projects_Score, Total_Score, Grade, Study_Hours_per_Week, Extracurricular_Activities, Internet_Access_at_Home, Parent_Education_Level, Family_Income_Level, Stress_Level (1-10), Sleep_Hours_per_Night
- Column Types:
  • Student_ID: object
  • First_Name: object
  • Last_Name: object
  • Email: object
  • Gender: object
  • Age: int64
  • Department: objec

PDF FILE INPUT

In [6]:
if __name__ == "__main__":
    chatbot()


🔹 RAG Chatbot - Upload Files or URLs
Enter file path or URL (or 'done' to finish): /content/Data Science Interview.pdf
✅ Processing /content/Data Science Interview.pdf ...
✅ Successfully processed /content/Data Science Interview.pdf.
Enter file path or URL (or 'done' to finish): done

✅ Index Built Successfully!


🔍 Enter your query (or 'exit' to quit):  WHAT IS A DATA INCIDENT IN HUMANITARIAN RESPONSE

📌 **Top Retrieved Contexts:**
WHAT IS A DATA INCIDENT IN HUMANITARIAN RESPONSE?
In the humanitarian sector, data incidents are events involving the management of data that have caused
harm or have the potential to cause harm to crisis affected populations, humanitarian organisations
and their operations, and other individuals or groups.
THE CENTRE FOR HUMANITARIAN DATA
GUIDANCE NOTE SERIES
DATA RESPONSIBILITY IN HUMANITARIAN ACTION
DATA INCIDENT MANAGEMENT
KEY TAKEAWAYS:
• Humanitarian data incidents are events involving the management of data that have caused
harm or have the potentia

IMAGE FILE INPUT


In [7]:
if __name__ == "__main__":
    chatbot()


🔹 RAG Chatbot - Upload Files or URLs
Enter file path or URL (or 'done' to finish): /content/Screenshot 2024-07-12 232612.png
✅ Processing /content/Screenshot 2024-07-12 232612.png ...
✅ Successfully processed /content/Screenshot 2024-07-12 232612.png.
Enter file path or URL (or 'done' to finish): done

✅ Index Built Successfully!


🔍 Enter your query (or 'exit' to quit): list there components if presnt

📌 **Top Retrieved Contexts:**
Digital Marketing Types

Content Marketing

Social Media Marketing ce =| A SEO Search Engine Optimization

E-mail Marketing a
Affiliate Marketing ®

Viral Marketing

  

j | Search Engine Marketing

ro Influencer Marketing
Digital Marketing Types

Content Marketing

Social Media Marketing ce =| A SEO Search Engine Optimization

E-mail Marketing a
Affiliate Marketing ®

Viral Marketing

  

j | Search Engine Marketing

ro Influencer Marketing
Digital Marketing Types

Content Marketing

Social Media Marketing ce =| A SEO Search Engine Optimization

E-mail Ma

VIDEO FILE INPUT

In [8]:
if __name__ == "__main__":
    chatbot()


🔹 RAG Chatbot - Upload Files or URLs
Enter file path or URL (or 'done' to finish): /content/videoplayback.mp4
✅ Processing /content/videoplayback.mp4 ...
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 118MiB/s]
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



✅ Successfully processed /content/videoplayback.mp4.
Enter file path or URL (or 'done' to finish): done

✅ Index Built Successfully!


🔍 Enter your query (or 'exit' to quit): explain it

📌 **Top Retrieved Contexts:**
Their ability to retain information over long sequences makes them superior to simple RNNs and tasks requiring long-term context.
Long short-term memory networks, or LSTMs, are a sophisticated type of RNN designed to address the vanishing gradient problem.
Unlike simple RNNs, LSTMs incorporate a more complex cell structure that allows them to remember information for extended periods.

📌 **Abstractive Summary:**
Long short-term memory networks, or LSTMs, are a sophisticated type of RNN designed to address the vanishing gradient problem. Their ability to retain information over long sequences makes them superior to simple RNNs and tasks requiring long-term context.

🔍 Enter your query (or 'exit' to quit): exit


DOCS INPUT FILE

In [9]:
if __name__ == "__main__":
    chatbot()


🔹 RAG Chatbot - Upload Files or URLs
Enter file path or URL (or 'done' to finish): /content/db.docx
✅ Processing /content/db.docx ...
✅ Successfully processed /content/db.docx.
Enter file path or URL (or 'done' to finish): done

✅ Index Built Successfully!


🔍 Enter your query (or 'exit' to quit): explain it

📌 **Top Retrieved Contexts:**
db.events.insert([
  {
    eventId: 1,
    name: "Tech Conference 2025",
    date: ISODate("2025-06-15"),
    location: "chennai",
    attendees: 500,
    organizer: "Tech Corporate",
    ticketsAvailable: 150
  },
  {
    eventId: 2,
    name: "Music Festival",
    date: ISODate("2025-07-20"),
    location: Coimbatore",
    attendees: 2000,
    organizer: "Vijay Antony",
    ticketsAvailable: 50
  },
  {
    eventId: 3,
    name: "AI Workshop",
    date: ISODate("2025-11-10"),
    location: "CIT",
    attendees: 100,
    organizer: "AIML Lab",
    ticketsAvailable: 20
  },
{
    eventId: 4,
    name: "Dance Marathon",
    date: ISODate("2025-09-24")

URL AS INPUT

In [10]:
if __name__ == "__main__":
    chatbot()


🔹 RAG Chatbot - Upload Files or URLs
Enter file path or URL (or 'done' to finish): https://www.geeksforgeeks.org/machine-learning-algorithms/
✅ Processing https://www.geeksforgeeks.org/machine-learning-algorithms/ ...
✅ Successfully processed https://www.geeksforgeeks.org/machine-learning-algorithms/.
Enter file path or URL (or 'done' to finish): done

✅ Index Built Successfully!


🔍 Enter your query (or 'exit' to quit): describe

📌 **Top Retrieved Contexts:**
For in-depth understanding : What is Ensemble Learning?
For in-depth understanding : Supervised multi-layer perceptron model –  What is perceptron?
These methods use a model of the environment to predict outcomes and help the agent plan actions by simulating potential results.

📌 **Abstractive Summary:**
These methods use a model of the environment to predict outcomes and help the agent plan actions by simulating potential results. These methods are called Ensemble Learning and Supervised multi-layer perceptron models. For in-de

YOUTUBE VIDEO LINK INPUT WITHOUT SUBTITLE

In [11]:
if __name__ == "__main__":
    chatbot()


🔹 RAG Chatbot - Upload Files or URLs
Enter file path or URL (or 'done' to finish): https://www.youtube.com/shorts/CJ_ZAmjyiQg
✅ Processing https://www.youtube.com/shorts/CJ_ZAmjyiQg ...
✅ Successfully processed https://www.youtube.com/shorts/CJ_ZAmjyiQg.
Enter file path or URL (or 'done' to finish): done

✅ Index Built Successfully!


🔍 Enter your query (or 'exit' to quit): explain it

📌 **Top Retrieved Contexts:**
This is most likely caused by:

You provided an invalid video id.
Error extracting subtitles from YouTube: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=https://www.youtube.com/shorts/CJ_ZAmjyiQg!
Do NOT run: `YouTubeTranscriptApi.get_transcript("https://www.youtube.com/watch?v=1234")`
Instead run: `YouTubeTranscriptApi.get_transcript("1234")`

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issu

YOUTUBE VIDEO WITH SUBTITLE INPUT

In [12]:
if __name__ == "__main__":
    chatbot()


🔹 RAG Chatbot - Upload Files or URLs
Enter file path or URL (or 'done' to finish): https://www.youtube.com/watch?v=VJHWOO2Ix_8&t=330s
✅ Processing https://www.youtube.com/watch?v=VJHWOO2Ix_8&t=330s ...
✅ Successfully processed https://www.youtube.com/watch?v=VJHWOO2Ix_8&t=330s.
Enter file path or URL (or 'done' to finish): done

✅ Index Built Successfully!


🔍 Enter your query (or 'exit' to quit): explain it

📌 **Top Retrieved Contexts:**
there are a few problems with recurrent neural network which is solved by this model named lstm which stands for long short term memory now in this video we will discuss what is lstm why do we need it and what is the problem with the recurrent neural network now this is the model which is a bit hard to understand and especially when you are looking at it or understanding it for the first time it does not quite make sense what is going on and same thing happened to me when i learned this concept for the first time i did not understand much so after go

MULTIPLE FILE PROCESSING
AT A TIME MULTIPLE FILES ARE INPUT


In [13]:
if __name__ == "__main__":
    chatbot()


🔹 RAG Chatbot - Upload Files or URLs
Enter file path or URL (or 'done' to finish): /content/Students_Grading_Dataset.csv
✅ Processing /content/Students_Grading_Dataset.csv ...
✅ Successfully processed /content/Students_Grading_Dataset.csv.
Enter file path or URL (or 'done' to finish): /content/Data Science Interview.pdf
✅ Processing /content/Data Science Interview.pdf ...
✅ Successfully processed /content/Data Science Interview.pdf.
Enter file path or URL (or 'done' to finish): /content/db.docx
✅ Processing /content/db.docx ...
✅ Successfully processed /content/db.docx.
Enter file path or URL (or 'done' to finish): done

✅ Index Built Successfully!


🔍 Enter your query (or 'exit' to quit): explain all

📌 **Top Retrieved Contexts:**
These scenarios demonstrate how to think about identifying causal chains that may create context-
specific data incidents.
4 IAPP, Is It an Incident or a Breach, How to Tell and Why It Matters, Mahmoud Sher-Jan (February 2017).
• The International Organizat